<h1>Chapter 2 | <b>EXTRA</b> data exercise | <b>Identifying Successful Football Managers in Brazilian Football</b></h1>
<p>In this notebook, I'll be writing the code for the aforementionred case study of the book. The main goal is to observe and learn how to structure the data according to our neeeds, which means applying the concept of <i>tidy data</i> aas well as merging different datasets to get the desired outcome.</p>
<p>Here, our question is: <i>who are the most successful football managers in <b>Brazil</b>?</p>


In [1]:
import os
import sys
import warnings
import yaml
import numpy as np
import pandas as pd
import seaborn as sns
from mizani.formatters import percent_format
from plotnine import *
import basedosdados as bd

warnings.filterwarnings("ignore")

In [2]:
# Current script folder path
current_path = os.getcwd()
dirname = f"{current_path}/data/"
config = f"{current_path}/config/"
# Location folders
data_in = f"{dirname}raw/"
data_out = f"{dirname}clean/"

func = f"{current_path.split('da_data_exercises')[0]}da_case_studies/ch00-tech_prep/"
sys.path.append(func)

In [3]:
with open(f"{config}config.yaml", "r") as f:
    project_id = yaml.safe_load(f)

In [4]:
from py_helper_functions import seq, color

<h2>2. Extract data from Basedosdados' GBQ</h2>

In [5]:
query = """
    SELECT 
  ano_campeonato AS season,
  data AS date,
  time_man AS home_team,
  time_vis AS away_team,
  tecnico_man AS manager_home,
  tecnico_vis AS manager_away,
  gols_man AS goals_home,
  gols_vis AS goals_away
    FROM `basedosdados.mundo_transfermarkt_competicoes.brasileirao_serie_a`
    WHERE ano_campeonato >= 2012
"""

In [6]:
df = bd.read_sql(query, billing_project_id=project_id["bd_id"])

Downloading: 100%|██████████| 4303/4303 [00:00<00:00, 4583.59rows/s]


In [7]:
df.head()

,season,date,home_team,away_team,manager_home,manager_away,goals_home,goals_away
0,2012,2012-05-27,Grêmio,Palmeiras,Vanderlei Luxemburgo,Luiz Felipe Scolari,1,0
1,2012,2012-07-01,Grêmio,Atlético-MG,Vanderlei Luxemburgo,Cuca,0,1
2,2012,2012-07-18,Grêmio,Sport Recife,Vanderlei Luxemburgo,Vágner Mancini,3,1
3,2012,2012-07-25,Grêmio,Fluminense,Vanderlei Luxemburgo,Abel Braga,1,0
4,2012,2012-08-15,Grêmio,Portuguesa,Vanderlei Luxemburgo,Geninho,1,2


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4303 entries, 0 to 4302
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   season        4303 non-null   Int64 
 1   date          4303 non-null   dbdate
 2   home_team     4303 non-null   object
 3   away_team     4303 non-null   object
 4   manager_home  4113 non-null   object
 5   manager_away  4113 non-null   object
 6   goals_home    4302 non-null   Int64 
 7   goals_away    4302 non-null   Int64 
dtypes: Int64(3), dbdate(1), object(4)
memory usage: 281.7+ KB


In [9]:
df[["goals_home", "goals_away"]].isna().value_counts()

goals_home  goals_away
False       False         4302
True        True             1
dtype: int64

In [10]:
df.dropna(subset=["goals_home", "goals_away"], inplace=True)

In [11]:
# Create "points_home" variable
df["points_home"] = df.apply(
    lambda row: 1 if row["goals_home"] == row["goals_away"] 
    else (3 if row["goals_home"] > row["goals_away"] else 0), 
    axis=1
    )

# Create "points_away" variable
df["points_away"] = df.apply(
    lambda row: 1 if row["goals_home"] == row["goals_away"] 
    else (3 if row["goals_home"] < row["goals_away"] else 0), 
    axis=1
    )

In [12]:
df.head()

,season,date,home_team,away_team,manager_home,manager_away,goals_home,goals_away,points_home,points_away
0,2012,2012-05-27,Grêmio,Palmeiras,Vanderlei Luxemburgo,Luiz Felipe Scolari,1,0,3,0
1,2012,2012-07-01,Grêmio,Atlético-MG,Vanderlei Luxemburgo,Cuca,0,1,0,3
2,2012,2012-07-18,Grêmio,Sport Recife,Vanderlei Luxemburgo,Vágner Mancini,3,1,3,0
3,2012,2012-07-25,Grêmio,Fluminense,Vanderlei Luxemburgo,Abel Braga,1,0,3,0
4,2012,2012-08-15,Grêmio,Portuguesa,Vanderlei Luxemburgo,Geninho,1,2,0,3


<p>Question: how can we group all managers and count their average number of points per match, regardless whether they played as home team or away team?</p>

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4302 entries, 0 to 4302
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   season        4302 non-null   Int64 
 1   date          4302 non-null   dbdate
 2   home_team     4302 non-null   object
 3   away_team     4302 non-null   object
 4   manager_home  4113 non-null   object
 5   manager_away  4113 non-null   object
 6   goals_home    4302 non-null   Int64 
 7   goals_away    4302 non-null   Int64 
 8   points_home   4302 non-null   int64 
 9   points_away   4302 non-null   int64 
dtypes: Int64(3), dbdate(1), int64(2), object(4)
memory usage: 511.3+ KB


In [58]:
df.dropna(inplace=True)

In [59]:
# Create a new DataFrame to store manager information
manager_df = pd.DataFrame()
manager_df["manager"] = df["manager_home"].append(df["manager_away"])
manager_df["points"] = df["points_home"].append(df["points_away"])
manager_df["season"] = df["season"]
manager_df["date"] = df["date"]

In [60]:
manager_df.tail()

,manager,points,season,date
4298,Jayme de Almeida,0,2014,2014-06-01
4299,Jorginho,0,2014,2014-08-10
4300,Ricardo Drubscky,0,2014,2014-05-31
4301,Marquinhos Santos,0,2014,2014-09-28
4302,Dorival Júnior,0,2014,2014-09-21


<p>Let's export this table to visualize in Tableau before aggregating further the data</p>

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4113 entries, 0 to 4302
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   season        4113 non-null   Int64 
 1   date          4113 non-null   dbdate
 2   home_team     4113 non-null   object
 3   away_team     4113 non-null   object
 4   manager_home  4113 non-null   object
 5   manager_away  4113 non-null   object
 6   goals_home    4113 non-null   Int64 
 7   goals_away    4113 non-null   Int64 
 8   points_home   4113 non-null   int64 
 9   points_away   4113 non-null   int64 
dtypes: Int64(3), dbdate(1), int64(2), object(4)
memory usage: 494.5+ KB


In [53]:
manager_df.to_csv(f"{data_out}brl_football_manager_success.csv", index=False)

In [43]:
grouped_manager_df = manager_df.groupby("manager").agg(
    avg_points_per_match = pd.NamedAgg(column="points", aggfunc="mean"),
    num_matches=pd.NamedAgg(column="points", aggfunc="count")).sort_values(by=["num_matches","avg_points_per_match"], ascending=[False, False])

In [41]:
grouped_manager_df.sort_values(by=["num_matches","avg_points_per_match"], ascending=[False, False])

,avg_points_per_match,num_matches
manager,,
Cuca,1.738007,271
Mano Menezes,1.586207,261
Dorival Júnior,1.518828,239
Marcelo Oliveira,1.518018,222
Vanderlei Luxemburgo,1.403846,208
...,...,...
Marcelo Serrano,0.000000,1
Narciso,0.000000,1
Robson Gomes,0.000000,1


In [44]:
grouped_manager_df.loc[
    grouped_manager_df["num_matches"] > 100
]

,avg_points_per_match,num_matches
manager,,
Cuca,1.738007,271
Mano Menezes,1.586207,261
Dorival Júnior,1.518828,239
Marcelo Oliveira,1.518018,222
Vanderlei Luxemburgo,1.403846,208
Vágner Mancini,1.260870,207
Guto Ferreira,1.210000,200
Enderson Moreira,1.225641,195
Renato Gaúcho,1.631579,190
